In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from utils.analyze_utils import getOveralStats

In [2]:
log_path = '../log.csv'
log_df = pd.read_csv(log_path, header=None, index_col=0)

In [3]:
# Create dataframes for each model

model_names = ['BSUV-Net-2']
model2df = {}

all_rows = ['category', 'scene', 'metric']
for model_name in model_names:
    model2df[model_name] = log_df.loc[model_name]

In [4]:
# Combine results into a single dataframe

comparison_df = log_df.loc[['category', 'scene', 'metric']]

for model_name, model_df in model2df.items():
    model_df = model_df.apply(pd.to_numeric, errors='coerce').replace(0, np.nan).median(skipna=True)
    model_df[0] = 0
    nan_idx = ~model_df.apply(np.isnan)
    model_df = model_df.loc[nan_idx]
    model_df.name = model_name
    comparison_df = comparison_df.append(model_df)

comparison_df

,1,2,3,4,5,6,7,8,9,10,...,151,152,153,154,155,156,157,158,159,0
0,,,,,,,,,,,,,,,,,,,,,
category,baseline,baseline,baseline,baseline,baseline,baseline,baseline,baseline,baseline,baseline,...,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,NaN
scene,highway,highway,highway,pedestrians,pedestrians,pedestrians,office,office,office,PETS2006,...,turbulence1,turbulence1,turbulence1,turbulence2,turbulence2,turbulence2,turbulence3,turbulence3,turbulence3,NaN
metric,FNR,Prec,F-score,FNR,Prec,F-score,FNR,Prec,F-score,FNR,...,FNR,Prec,F-score,FNR,Prec,F-score,FNR,Prec,F-score,NaN
BSUV-Net-2,0.00768751,0.998779,0.995535,0.12727,0.981341,0.923854,0.0119327,0.994457,0.991252,0.0247795,...,0.416979,0.931322,0.717116,0.0593145,0.973678,0.956897,0.262014,0.983667,0.843297,0.0


In [5]:
# Show the results

metric = 'F-score'
group_by = 'category'

metric_locs = comparison_df.iloc[2].str.contains(metric)
metric_locs = metric_locs.fillna(True)
metric_locs[0] = True
metric_df = comparison_df.loc[:, metric_locs].transpose()
metric_df[[*model_names]] = metric_df[[*model_names]].apply(pd.to_numeric)


if metric=='FNR':
    cat_based_df = 1-metric_df.groupby([group_by]).mean()
else:
    cat_based_df = metric_df.groupby([group_by]).mean()
cat_based_df.head(20)


cat_based_df.loc['MEAN'] = cat_based_df.mean()
cat_based_df = cat_based_df.round(4)
                      
cat_based_df

,BSUV-Net-2
category,
PTZ,0.6920
badWeather,0.8971
baseline,0.9708
cameraJitter,0.9170
dynamicBackground,0.9081
intermittentObjectMotion,0.8306
lowFramerate,0.8401
nightVideos,0.6326
shadow,0.9814
